### Application of the Bayesian Ranking Algorithm
Toy example: we are given 3 coins, each of them can either result in head (win) or tail (loss). We know that only one of them is fair, that is, $p(x=head)=0.5$: we have multiple objectives:
- find the fair coin, 
- rank all three coins by probability of success,
- win as many times as possible, i.e. toss more frequently the best-rewarding coin. 

Instead of tossing one coin at a time in order to collect data and rank them at the end, we're going to rank them by playing and updating the posterior distributions, sampling random points from them.\
This scenario can be easily translated into a recommender system, where, instead of win/loss, we have a certain score. This score could be: 
- Click-Through Rate (CTR), defined as $\frac{N_c}{N}$, where $N_c$ is the number of times the item has been clicked and $N$ is the total number of appearences,
- User ratings: in this case it could be useful to assign binary scores to each rating value.

**IDEA** behind Bayesian Ranking: we do NOT rank by the average value, but we instead use the posterior distribution of this parameter to sample random points. That is the strength of Bayesian statistics: we can work with *distributions*, we end up in a non-deterministic frame and we can compute uncertainties.

This approach also allows to automatically manage the **Explore-Exploit dilemma**, since the least-explored distribution (the one with the fewer samples) is favoured by the randomness of the sampling procedure.

For this type of Bernoullian situation, we can refer to the following relations:
$$p(\pi|X) = const \times \pi^{\alpha' - 1}(1-\pi)^{\beta' - 1}$$
$$\alpha' = \alpha + \sum^N_{i=1}X_i$$
$$\beta' = \beta + N - \sum^{N}_{i=1}X_i$$

This means that the expected value of the original Bernoulli distribution is $\beta$-distributed. We want to retrieve exactly $\pi$ by means of its posterior $p(\pi|X)$.

In [1]:
# Import the usual libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import clear_output

# Import the beta distribution from scipy.stats
from scipy.stats import beta

# Create a class that simulates the coin tosses and updates the posterior distributions using Bayesian Ranking
class Coin(object):
    # Initialize the class with the flat posterior probability
    # p is an array containing the true probabilities of the coins
    def __init__(self, p=[]):
        self.p = p
        self.alpha = 1
        self.beta = 1

    # Simulate a coin toss based on the true probability
    def toss(self):
        return np.random.rand() < self.p

    # Sample a random value from the posterior distribution
    def sample(self):
        return np.random.beta(self.alpha, self.beta)

    # Update the posterior distribution based on the outcome of a coin toss
    def update(self, x):
        if x:
            self.alpha += x
        else:
            self.beta += 1 - x

    # Create a plot of the posterior distribution
    def plot(self, coins, n):
        x = np.linspace(0, 1, 500) # Create a list of 1000 x values between 0 and 1
        # Create a plot of the posterior distribution for each coin
        for c in coins:
            y = beta.pdf(x, c.alpha, c.beta)
            plt.plot(x, y, label=f'True p = {c.p}')
        plt.legend()
        plt.title(f'Posterior Distribution after {n} trials')
        plt.xlabel('Probability of heads')
        plt.ylabel('Density')
        plt.grid(linestyle='--', alpha=0.5)
        plt.show();

    # Simulate a series of coin tosses and update the posterior distribution
    def simulate(self, n):
        # Create a list of 3 coins
        coins = [Coin(true_p) for true_p in self.p]
        for i in range(n):
            # Define the maximum sample value (inside the loop because we update the samples every time we toss a coin)
            maxsample = -1
            # Create a list of sample values
            samples = []
            for coin in coins:
                # Sample a random value from the posterior distribution: this will be used for ranking
                sample = coin.sample()
                samples.append(sample)
                # Identify the coin with the maximum sample value (this is done do minimize the sub-optimal coin tosses)
                if sample > maxsample:
                    maxsample = sample
                    maxcoin = coin
            # Update the posterior distribution of the coin with the maximum sample value
            maxcoin.update(maxcoin.toss())
            # Every n coin tosses, plot the posterior distributions
            if i in [5, 10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 9999]:
                # Print the 3 samples
                print(samples)
                # Clear the plot and wait for user input to continue
                self.plot(coins, i)
                c = input('Continue?')
                if c == 'n':
                    break
                clear_output(wait=False)
                
        return print('Simulation completed!')

In [2]:
# Simulate 10000 coin tosses
experiment = Coin(p=[0.2, 0.5, 0.75])
experiment.simulate(10000)

Simulation completed!
